We decided not to use this notebook due to the prohibitive time complexity of run_finbert()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import torch

# Load pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

In [ ]:
def sliding_window_text(text, max_length=510):
    # Split the text into words
    words = text.split(' ')
    
    chunks = []
    i = 0
    while i < len(words):
        chunk = ''
        while i < len(words) and len(tokenizer.tokenize(chunk + ' ' + words[i])) <= max_length:
            chunk += ' ' + words[i]
            i += 1
        chunks.append(chunk)
        
    return chunks

def get_transcripts(filepath, transcript_column="transcript"):
    """Returns a list consisting of all transcripts at given filepath."""
    df = pd.read_csv(filepath)
    transcripts = df[transcript_column].tolist()
    return transcripts

def run_finbert(transcripts):
    final_scores = []
    i = 0
    length = len(transcripts)

    for transcript in transcripts:
        sliding_chunks = sliding_window_text(transcript)

        scores = [0, 0, 0]  # [negativity, neutrality, positivity]

        for chunk in sliding_chunks:
            # Encode the text snippet and convert to torch tensor
            inputs = tokenizer.encode_plus(chunk, return_tensors='pt')

            # Get model outputs
            outputs = model(**inputs)

            # The sentiment scores are the elements of the first output
            sentiment_scores = outputs[0][0].detach().numpy()

        # Add the sentiment scores to the cumulative scores
        scores = [sum(x) for x in zip(scores, sentiment_scores)]
        
        # Calculate synthetic sentiment score
        synthetic_score = (scores[0] - scores[2]) / (scores[0] + scores[2] + 1)
        
        final_scores.append(synthetic_score)
        if i%10 == 0:
            print(f"Done: {i/length}")
            save_intermediate_results(final_scores, 'FINBERT-intermediate_results.csv')
        
        i += 1

    return final_scores

def save_intermediate_results(scores, filename):
    df = pd.DataFrame({'sentiment_score': scores})
    df.to_csv(filename, index=False)

sentiment_scores = run_finbert(get_transcripts("..\data\\final_dataset.csv"))

In [ ]:
sentiment_scores

In [ ]:
# Normalize sentiment scores to be between -1 and 1:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

sentiment_scores = np.array(sentiment_scores)

scaler = MinMaxScaler(feature_range = (-1, 1))
normalized_scores = scaler.fit_transform(sentiment_scores.reshape(-1, 1))

normalized_scores

Optional:

In [ ]:
"""
Add normalized_scores to final_dataset.csv: (We never got this far).
"""

whole_dataset = pd.read_csv("..\data\\final_dataset.csv")

whole_dataset['finbert'] = normalized_scores

whole_dataset.to_csv("..\data\\final_dataset.csv", index=False)


no_transcripts = pd.read_csv("..\data\\no_transcript_final_dataset.csv")
whole_dataset.drop(columns="transcript", inplace=True)
# df
no_transcripts.to_csv("..\data\\no_transcript_final_dataset.csv",index=False)